<a href="https://colab.research.google.com/github/ritamgh/DLT-lab/blob/main/Week14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================================
# Lab 14: Transfer Learning - Feature Extractor on CIFAR-10
# ==========================================================
!pip -q install torch torchvision torchinfo

import torch, torch.nn as nn, torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from torchinfo import summary
from time import time

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

# --- Data (CIFAR-10 -> 224, ImageNet stats)
img_size = 224
batch_size = 128
num_classes = 10

weights18 = models.ResNet18_Weights.IMAGENET1K_V1
mean, std = weights18.meta.get('mean', (0.485,0.456,0.406)), weights18.meta.get('std', (0.229,0.224,0.225))

tf_train = transforms.Compose([
    transforms.Resize((img_size,img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(img_size, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])
tf_val = transforms.Compose([
    transforms.Resize((img_size,img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

trainset = datasets.CIFAR10("/content/data", train=True, download=True, transform=tf_train)
valset   = datasets.CIFAR10("/content/data", train=False, download=True, transform=tf_val)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

# --- Model
backbone = models.resnet18(weights=weights18)
for p in backbone.parameters():
    p.requires_grad = False  # freeze all

in_features = backbone.fc.in_features
backbone.fc = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(in_features, num_classes)
)
model = backbone.to(device).train()
print(summary(model, input_size=(1,3,img_size,img_size), verbose=0))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)
epochs = 5  # increase for stronger results

def evaluate():
    model.eval()
    correct, total, val_loss = 0, 0, 0.0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            loss = criterion(logits, y)
            val_loss += loss.item()*x.size(0)
            pred = logits.argmax(1)
            correct += (pred==y).sum().item()
            total += y.size(0)
    model.train()
    return val_loss/total, correct/total

best_acc = 0.0
for ep in range(1, epochs+1):
    t0 = time()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
    vloss, vacc = evaluate()
    best_acc = max(best_acc, vacc)
    print(f"Epoch {ep}/{epochs} | val_loss={vloss:.4f} | val_acc={vacc*100:.2f}% | time={(time()-t0):.1f}s")

print(f"Done. Best Val Acc (frozen backbone): {best_acc*100:.2f}%")

# --- Optional: Fine-tune last block + head
for m in [model.layer4, model.fc]:
    for p in m.parameters(): p.requires_grad = True

optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-4)
epochs_ft = 3
for ep in range(1, epochs_ft+1):
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        loss = criterion(model(x), y)
        loss.backward()
        optimizer.step()
    vloss, vacc = evaluate()
    print(f"[FT] Epoch {ep}/{epochs_ft} | val_loss={vloss:.4f} | val_acc={vacc*100:.2f}%")

print("Transfer learning complete.")

Device: cuda


100%|██████████| 170M/170M [00:03<00:00, 48.3MB/s]


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 157MB/s]


Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 10]                   --
├─Conv2d: 1-1                            [1, 64, 112, 112]         (9,408)
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         (128)
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           (36,864)
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           (128)
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           (36,864)
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           (128)
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]   